<a href="https://colab.research.google.com/github/DeivisDervinis/EAPDA/blob/main/colab/EAPDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install PyAudio
!pip install pydub

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1).
libportaudiocpp0 is already the newest version (19.6.0-1).
portaudio19-dev is already the newest version (19.6.0-1).
libasound2-dev is already the newest version (1.1.3-5ubuntu0.5).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following packages were automatically installed and are no longer required:
  libpcre2-16-0 libpcre2-32-0 libpcre2-dev libpcre2-posix0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 55 not upgraded.


In [ ]:
!mkdir models
!wget http://github.com/DeivisDervinis/EAPDA/raw/main/colab/models/audio.hdf5 -O models/audio.hdf5

mkdir: cannot create directory ‘models’: File exists
URL transformed to HTTPS due to an HSTS policy
--2021-03-05 23:15:52--  https://github.com/DeivisDervinis/EAPDA/raw/main/colab/models/audio.hdf5
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DeivisDervinis/EAPDA/main/colab/models/audio.hdf5 [following]
--2021-03-05 23:15:52--  https://raw.githubusercontent.com/DeivisDervinis/EAPDA/main/colab/models/audio.hdf5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5368576 (5.1M) [application/octet-stream]
Saving to: ‘models/audio.hdf5’

models/audio.hdf5   100%[===================>]   5.12M  --.-KB/

In [ ]:
!mkdir data
!wget http://github.com/DeivisDervinis/EAPDA/raw/main/colab/data/f13.wav -O data/f13.wav
!wget http://github.com/DeivisDervinis/EAPDA/raw/main/colab/data/h10.wav -O data/h10.wav
!wget http://github.com/DeivisDervinis/EAPDA/raw/main/colab/data/sa01.wav -O data/sa01.wav
!wget http://github.com/DeivisDervinis/EAPDA/raw/main/colab/data/su02.wav -O data/su02.wav

URL transformed to HTTPS due to an HSTS policy
--2021-03-05 23:17:27--  https://github.com/DeivisDervinis/EAPDA/raw/main/colab/data/f13.wav
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DeivisDervinis/EAPDA/main/colab/data/f13.wav [following]
--2021-03-05 23:17:28--  https://raw.githubusercontent.com/DeivisDervinis/EAPDA/main/colab/data/f13.wav
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274628 (268K) [audio/wav]
Saving to: ‘data/f13.wav’

data/f13.wav        100%[===================>] 268.19K  --.-KB/s    in 0.02s   

2021-03-05 23:17:28 (10.7 MB/s) - ‘data/f13.wav’ saved [274628/274628]


In [ ]:
## Basics ##
import time
import os
import numpy as np
## Audio Preprocessing ##
import pyaudio
import wave
import librosa
from scipy.stats import zscore

## Time Distributed CNN ##
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, TimeDistributed
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten
from tensorflow.keras.layers import LSTM

In [ ]:
class speechEmotionRecognition:

    ###
    ##Voice recording function
    ##

    def __init__(self, subdir_model=None):

        # Load prediction model
        if subdir_model is not None:
            self._model = self.build_model()
            self._model.load_weights(subdir_model)

        # Emotion encoding
        self._emotion = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Neutral', 5:'Sad', 6:'Surprise'}


    ### Computing Mel-Spectrogram
    def mel_spectrogram(self, y, sr=16000, n_fft=512, win_length=256, hop_length=128, window='hamming', n_mels=128, fmax=4000):

        # Compute spectogram
        mel_spect = np.abs(librosa.stft(y, n_fft=n_fft, window=window, win_length=win_length, hop_length=hop_length)) ** 2

        # Compute mel spectrogram
        mel_spect = librosa.feature.melspectrogram(S=mel_spect, sr=sr, n_mels=n_mels, fmax=fmax)

        # Compute log-mel spectrogram
        mel_spect = librosa.power_to_db(mel_spect, ref=np.max)

        return np.asarray(mel_spect)



    # Audio framing
    def frame(self, y, win_step=64, win_size=128):

        # Number of frames
        nb_frames = 1 + int((y.shape[2] - win_size) / win_step)

        # Framming
        frames = np.zeros((y.shape[0], nb_frames, y.shape[1], win_size)).astype(np.float16)
        for t in range(nb_frames):
            frames[:,t,:,:] = np.copy(y[:,:,(t * win_step):(t * win_step + win_size)]).astype(np.float16)

        return frames


    # Builds TDCNN
    def build_model(self):

        # Clear Keras session
        K.clear_session()

        # Define input
        input_y = Input(shape=(5, 128, 128, 1), name='Input_MELSPECT')

        # First LFLB (local feature learning block)
        y = TimeDistributed(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_1_MELSPECT')(input_y)
        y = TimeDistributed(BatchNormalization(), name='BatchNorm_1_MELSPECT')(y)
        y = TimeDistributed(Activation('elu'), name='Activ_1_MELSPECT')(y)
        y = TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'), name='MaxPool_1_MELSPECT')(y)
        y = TimeDistributed(Dropout(0.2), name='Drop_1_MELSPECT')(y)

        # Second LFLB (local feature learning block)
        y = TimeDistributed(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_2_MELSPECT')(y)
        y = TimeDistributed(BatchNormalization(), name='BatchNorm_2_MELSPECT')(y)
        y = TimeDistributed(Activation('elu'), name='Activ_2_MELSPECT')(y)
        y = TimeDistributed(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'), name='MaxPool_2_MELSPECT')(y)
        y = TimeDistributed(Dropout(0.2), name='Drop_2_MELSPECT')(y)

        # Third LFLB (local feature learning block)
        y = TimeDistributed(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_3_MELSPECT')(y)
        y = TimeDistributed(BatchNormalization(), name='BatchNorm_3_MELSPECT')(y)
        y = TimeDistributed(Activation('elu'), name='Activ_3_MELSPECT')(y)
        y = TimeDistributed(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'), name='MaxPool_3_MELSPECT')(y)
        y = TimeDistributed(Dropout(0.2), name='Drop_3_MELSPECT')(y)

        # Fourth LFLB (local feature learning block)
        y = TimeDistributed(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_4_MELSPECT')(y)
        y = TimeDistributed(BatchNormalization(), name='BatchNorm_4_MELSPECT')(y)
        y = TimeDistributed(Activation('elu'), name='Activ_4_MELSPECT')(y)
        y = TimeDistributed(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'), name='MaxPool_4_MELSPECT')(y)
        y = TimeDistributed(Dropout(0.2), name='Drop_4_MELSPECT')(y)

        # Flat
        y = TimeDistributed(Flatten(), name='Flat_MELSPECT')(y)

        # LSTM layer
        y = LSTM(256, return_sequences=False, dropout=0.2, name='LSTM_1')(y)

        # Fully connected
        y = Dense(7, activation='softmax', name='FC')(y)

        # Build final model
        model = Model(inputs=input_y, outputs=y)

        return model



    # Predict speech emotion over time
    def predict_emotion_from_file(self, filename, chunk_step=16000, chunk_size=40000, predict_proba=False, sample_rate=16000):

        # Read audio file
        y, sr = librosa.core.load(filename, sr=sample_rate, offset=0.5)

        # Split audio signals into chunks
        chunks = self.frame(y.reshape(1, 1, -1), chunk_step, chunk_size)

        # Reshape chunks
        chunks = chunks.reshape(chunks.shape[1],chunks.shape[-1])

        # Z-normalization
        y = np.asarray(list(map(zscore, chunks)))

        # Compute mel spectrogram
        mel_spect = np.asarray(list(map(self.mel_spectrogram, y)))

        # Time distributed Framing
        mel_spect_ts = self.frame(mel_spect)

        # Build X for time distributed CNN
        X = mel_spect_ts.reshape(mel_spect_ts.shape[0],
                                    mel_spect_ts.shape[1],
                                    mel_spect_ts.shape[2],
                                    mel_spect_ts.shape[3],
                                    1)

        # Predict emotion
        if predict_proba is True:
            predict = self._model.predict(X)
        else:
            predict = np.argmax(self._model.predict(X), axis=1)
            predict = [self._emotion.get(emotion) for emotion in predict]

        # Clear Keras session
        K.clear_session()

        # Predict timestamp
        timestamp = np.concatenate([[chunk_size], np.ones((len(predict) - 1)) * chunk_step]).cumsum()
        timestamp = np.round(timestamp / sample_rate)

        return [predict, timestamp]



    #Export emotions
    def prediction_to_csv(self, predictions, filename, mode='w'):

        # Write emotion in filename
        with open(filename, mode) as f:
            if mode == 'w':
                f.write("EMOTIONS"+'\n')
            for emotion in predictions:
                f.write(str(emotion)+'\n')
            f.close()


In [ ]:
# Record the voice of the user

$SER = speechEmotionRecognition()

# Set the duration of the recording
#recording_duration = 15
#rec_sub_dir = os.path.join('data','voice_recording.wav')
#SER.voice_recording(rec_sub_dir, duration=recording_duration)

OSError: ignored

In [ ]:
from pydub import AudioSegment
from pydub.playback import play
import math

# Sets the duration of audio
audio_duration = math.floor(librosa.get_duration(filename='data/sa01.wav'))
# Starting slice and end slice values
n = 0
nSpan = 3

# List of sliced segments
listOfSegNames = []

for i in range(0, (audio_duration + 1) - nSpan):   
    t1 = n * 1000
    t2 = (n + nSpan) * 1000
    
    oldAudio = AudioSegment.from_wav("data/sa01.wav")
    newAudio = oldAudio[t1:t2]
    seg_name = "data/seg_" + str(n) +".wav"
    listOfSegNames.append(seg_name)
    
    #listOfSegments.append(newAudio)
    newAudio.export(seg_name, format="wav")    
    n += 1

#play(listOfSegments[5])


In [ ]:
from collections import Counter

# Function that finds the most common occuring emotion
def find_state_emotion(listOfEmotions):
        
        count = Counter(listOfEmotions)
        max_val = max(count.values())
        
        return sorted(key for key, value in count.items() if value == max_val)
        

In [ ]:
model_sub_dir = os.path.join('models', 'audio.hdf5')

SER = speechEmotionRecognition(model_sub_dir)

segmentEmoList = []

emotionFoundFlag = False
finalEmotion = ""


# Run prediction for each segment
for k in range(0, len(listOfSegNames)):
    rec_sub_dir = listOfSegNames[k]
    
    # Predict emotion in voice at each time step
    step = 1 # in sec
    sample_rate = 16000 # in kHz
    emotions, timestamp = SER.predict_emotion_from_file(rec_sub_dir, chunk_step=step*sample_rate)
    
    major_emotion = max(set(emotions), key=emotions.count)
    segmentEmoList.append(major_emotion)
    
    
    finalEmotion = find_state_emotion(segmentEmoList)
    if len(finalEmotion) == 1:
        emotionFoundFlag = True
        break
            


if emotionFoundFlag == True:
    print("Emotion of the voice recording: "+str(finalEmotion))


ValueError: ignored